In [32]:
import pandas as pd 
import os
import numpy as np

os.chdir(r'C:\Users\nicho\OneDrive\Desktop\Bitcoin Regresssion Project')
BTC = pd.read_csv(r'BTC-USD.csv' , parse_dates = ['Date'], index_col=['Date'])
SNP = pd.read_csv(r'Standard&PoorsHistorical.csv', parse_dates = ['Date'], index_col=['Date'])
Margin = pd.read_csv(r'margin-statistics.csv')
CPI = pd.read_csv(r'CPIHistoricalData.csv', parse_dates = ['Date'], index_col= ['Date'])
BTC.head(), Margin.head(), SNP.head(), CPI.head()

(            Symbol     Open     High      Low    Close  Volume BTC  \
 Date                                                                 
 2020-04-10  BTCUSD  7315.25  7315.25  7315.25  7315.25        0.00   
 2020-04-09  BTCUSD  7369.60  7378.85  7115.04  7315.25     2237.13   
 2020-04-08  BTCUSD  7201.81  7432.23  7152.80  7369.60     2483.60   
 2020-04-07  BTCUSD  7336.96  7468.42  7078.00  7201.81     2333.34   
 2020-04-06  BTCUSD  6775.21  7369.76  6771.01  7336.96     3727.47   
 
              Volume USD  
 Date                     
 2020-04-10         0.00  
 2020-04-09  16310014.53  
 2020-04-08  18138080.27  
 2020-04-07  17047120.32  
 2020-04-06  26533750.17  ,
   Year-Month Debit Balances in Customers' Securities Margin Accounts  \
 0    2021-11                                            918,598        
 1    2021-10                                            935,862        
 2    2021-09                                            903,117        
 3    2021-08      

In [33]:
#Dropping irrelevant features

Drop = ['Symbol', 'Open', 'High', 'Low', 'Volume BTC']
BTCDrop = BTC.drop(Drop, axis = 1)

Drop1 = ['Volume', 'Open', 'High', 'Low']
SNPDrop = SNP.drop(Drop1, axis = 1)


Drop3 = ['Open', 'High', 'Low', 'Volume']
CPIDrop = CPI.drop(Drop3, axis = 1)

In [34]:
#Renaming Columns
BTCDrop.rename(columns = {'Close': 'BTC', 
                          'Volume USD': 'Liquidity'}, inplace = True)

SNPDrop.rename(columns = {'Close/Last': 'SNP close'}, inplace = True)

CPIDrop.rename(columns = {'Close/Last': 'CPI close'}, inplace = True)

In [35]:
#Converting feature samples to monthly 
BTCMonthly = BTCDrop.resample('M').median()
CPIMonthly = CPIDrop.resample('M').median()
SNPMonthly = SNPDrop.resample('M').median()

#Setting index back to default
BTCMonthly.reset_index(inplace = True)
SNPMonthly.reset_index(inplace = True)
CPIMonthly.reset_index(inplace = True)

#Changing layout to use as key for merge
BTCMonthly['Date'] = BTCMonthly['Date'].dt.strftime('%Y-%m')
SNPMonthly['Date'] = SNPMonthly['Date'].dt.strftime('%Y-%m')
CPIMonthly['Date'] = CPIMonthly['Date'].dt.strftime('%Y-%m')



# Creating 2 Dataframes for testing to understand if larger data makes a difference in the predictability power

In [49]:
#Dataframe with Margin feature
MarginMerged = pd.concat([Margin, BTCMonthly, SNPMonthly, CPIMonthly], axis=1, join='inner')

MarginMerged.drop(['Free Credit Balances in Customers\' Cash Accounts', 'Date', 
             'Free Credit Balances in Customers\' Securities Margin Accounts'], axis = 1, inplace = True)

In [108]:
#Cleaning comma from feature
NewDebit = MarginMerged['Debit Balances in Customers\' Securities Margin Accounts']
NewDebit = [col.replace(',', '') for col in NewDebit]
MarginMerged['Debit Balances in Customers\' Securities Margin Accounts'] = NewDebit

In [106]:
#Dataframe without Margin feature
Merged = pd.concat([BTCDrop, SNPDrop, CPIDrop], axis=1, join='inner')

In [100]:
#Visualization


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 6 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   Year-Month                                               55 non-null     object 
 1   Debit Balances in Customers' Securities Margin Accounts  55 non-null     object 
 2   BTC                                                      55 non-null     float64
 3   Liquidity                                                55 non-null     float64
 4   SNP close                                                55 non-null     float64
 5   CPI close                                                55 non-null     float64
dtypes: float64(4), object(2)
memory usage: 2.7+ KB
